# Backups

***You will need to have followed the steps in the 00-CreateContainers notebook to use this notebook***

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

dbatools is **awesome** at performing backups :-)

The next block sets the variables for tje instances and folder paths for this Notebook and checks the connection - Refer to the first notebook for any issues

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

 Creating connection to the containers


We have a connection to the containers


## Investigate a directory using the SQL Service Account

We can investigate the file system from the viewpoint of the SQL Service Account using `Get-DbaFile` We are going to use that to show the files in the directory but this can be really useful when investigating errors with file system access, for example when backups fail or when reading from shares. This command will try ot access the path from teh SQL Instance using the SQL Account which means that you can troubleshoot the error and also that users do not need access to directories holding backups or client files for example  
If we look in the Directory that we created in the create container notebook using the command below

In [2]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename                                  SqlInstance


--------                                  -----------


/var/opt/mssql/backups/SQL1\dummyfile.txt b7ee5654561f


You can see that all there is in the directory is a dummy file that we created earlier when we set up the directories  

## Check the last time that the databases were backed up

Knowing the last time a database was backed up is often useful information and with dbatools it can be retrieved easily. (NB - I use Warning action silently continue to hide the backups from multiple forks warnings that I have created when I created the container and havent fixed yet!)

In [3]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

ComputerName InstanceName SqlInstance  Database           LastFullBackup          LastDiffBackup


------------ ------------ -----------  --------           --------------          --------------   


localhost    MSSQLSERVER  b7ee5654561f AdventureWorks2017 2019-12-21 14:42:01.000 2019-12-21 14:42…


localhost    MSSQLSERVER  b7ee5654561f master             2019-12-21 14:41:56.000                  


localhost    MSSQLSERVER  b7ee5654561f model              2019-12-21 14:41:56.000                  


...

localhost    MSSQLSERVER  31f2db5347e2 master             2019-12-21 14:41:56.000                  


localhost    MSSQLSERVER  31f2db5347e2 model              2019-12-21 14:41:56.000                  


localhost    MSSQLSERVER  31f2db5347e2 msdb               2019-12-21 14:41:57.000                  


localhost    MSSQLSERVER  31f2db5347e2 Northwind          2019-12-21 14:42:01.000 2019-12-21 14:42…


localhost    MSSQLSERVER  31f2db5347e2 pubs               2019-12-21 14:42:02.000 2019-12-21 14:42…


The databases were last backed up on the 21st December 2019. Thats not so clever!

## Docker requires permissions

You will get errors for the command below if the account that you have used to allow Docker to access your drives does not have permissions to the folder that we have created. [You can see the instructions here](https://docs.docker.com/docker-for-windows/)

You should add the docker account with full control over your dbatoolsdemo directory in your User Profile directory if you get errors below.

![DockerCompose](.\images\dbatoolsdemopermissions.png )

Its annoying but you can give the docker account modify permissions to your documents folder and it will inherited

![DockerCompose](.\images\documentsdirectorypermissions.png )



## Perform a backup of the entire instance

With one line of code we can quickly backup an entire instance.

In [4]:
Backup-DbaDatabase  -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL1\AdventureWorks2017_20200617
    Progress: 0 %                                                                                   
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL1\AdventureWorks2017_20200617
    Progress: 1 %                                                                                   
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL1\AdventureWorks2017_20200617
    Progress: 36 %                                                                                  
    [ooooooooooooooooooooooooooooooo                                                         ]      
                                                                                                    

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL1\AdventureWorks2017_20200617
    Progress: 86 %                                                                                  
    [ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo             ]      
                                                                                                    

                                                                                                    
 Backing up database master to c:\var\opt\mssql\backups\SQL1\master_202006171950.bak                
    Progress: 0 %                                                                                   
    [                                                                                        ]      
                                                                                                    

...

b7ee5654561f master             Full 6.71 MB   Disk       2020-06-17 18:50:18.000 00:00:00 2020-06…


b7ee5654561f model              Full 2.59 MB   Disk       2020-06-17 18:50:18.000 00:00:00 2020-06…


b7ee5654561f msdb               Full 16.09 MB  Disk       2020-06-17 18:50:19.000 00:00:00 2020-06…


b7ee5654561f Northwind          Full 6.71 MB   Disk       2020-06-17 18:50:19.000 00:00:00 2020-06…


b7ee5654561f pubs               Full 4.52 MB   Disk       2020-06-17 18:50:19.000 00:00:01 2020-06…


All of the databases are backed up - Lets have a look in the directory

In [5]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename                                                        SqlInstance


--------                                                        -----------


/var/opt/mssql/backups/SQL1\AdventureWorks2017_202006171950.bak b7ee5654561f


/var/opt/mssql/backups/SQL1\dummyfile.txt                       b7ee5654561f


/var/opt/mssql/backups/SQL1\master_202006171950.bak             b7ee5654561f


/var/opt/mssql/backups/SQL1\model_202006171950.bak              b7ee5654561f


/var/opt/mssql/backups/SQL1\msdb_202006171950.bak               b7ee5654561f


/var/opt/mssql/backups/SQL1\Northwind_202006171950.bak          b7ee5654561f


/var/opt/mssql/backups/SQL1\pubs_202006171950.bak               b7ee5654561f


If you want to have a look in the mounted volume on your laptop 

In [6]:
Get-ChildItem "$FolderPath\SQL1"

    Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL1


Mode                 LastWriteTime         Length Name


----                 -------------         ------ ----


-a---          17/06/2020    19:50       50442240 AdventureWorks2017_202006171950.bak


-a---          17/06/2020    19:16              0 dummyfile.txt


-a---          17/06/2020    19:50        1486848 master_202006171950.bak


-a---          17/06/2020    19:50         450560 model_202006171950.bak


-a---          17/06/2020    19:50        2732032 msdb_202006171950.bak


-a---          17/06/2020    19:50         864256 Northwind_202006171950.bak


-a---          17/06/2020    19:50         524288 pubs_202006171950.bak


## Backup to a directory for each database

Rob - I hear you cry, I have 2000 databases, I dont want them all backing up to the same directory - Can dbatools help me there ?

Sure, just add the `-CreateFolder` switch.

Lets demonstrate with SQL2

Theres nothing up my sleeves, lets look at the SQL2 folder


In [7]:
Get-DbaFile -SqlInstance $SQL2 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL2

Filename                                  SqlInstance


--------                                  -----------


/var/opt/mssql/backups/SQL2\dummyfile.txt 31f2db5347e2


Only the dummy file! 

### Backup the databases

In [8]:
Backup-DbaDatabase -SqlInstance $SQL2 -SqlCredential $SqlCredential -CreateFolder -Path /var/opt/mssql/backups/SQL2 -CompressBackup -WarningAction SilentlyContinue

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL2\AdventureWorks2017\Adventur
    Progress: 0 %                                                                                   
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL2\AdventureWorks2017\Adventur
    Progress: 1 %                                                                                   
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL2\AdventureWorks2017\Adventur
    Progress: 42 %                                                                                  
    [oooooooooooooooooooooooooooooooooooo                                                    ]      
                                                                                                    

                                                                                                    
 Backing up database AdventureWorks2017 to c:\var\opt\mssql\backups\SQL2\AdventureWorks2017\Adventur
    Progress: 100 %                                                                                 
    [oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo]      
                                                                                                    

                                                                                                    
 Backing up database master to c:\var\opt\mssql\backups\SQL2\master\master_202006171951.bak         
    Progress: 0 %                                                                                   
    [                                                                                        ]      
                                                                                                    

...

31f2db5347e2 master             Full 6.71 MB   Disk       2020-06-17 18:51:06.000 00:00:00 2020-06…


31f2db5347e2 model              Full 2.59 MB   Disk       2020-06-17 18:51:07.000 00:00:00 2020-06…


31f2db5347e2 msdb               Full 16.09 MB  Disk       2020-06-17 18:51:07.000 00:00:00 2020-06…


31f2db5347e2 Northwind          Full 6.71 MB   Disk       2020-06-17 18:51:08.000 00:00:00 2020-06…


31f2db5347e2 pubs               Full 4.52 MB   Disk       2020-06-17 18:51:08.000 00:00:01 2020-06…


## have a look in the directory

In [9]:
Get-ChildItem "$FolderPath\SQL2" -Recurse

    Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL2


Mode                 LastWriteTime         Length Name


----                 -------------         ------ ----


d----          17/06/2020    19:51                AdventureWorks2017


d----          17/06/2020    19:51                master


...

-a---          17/06/2020    19:51         864256 Northwind_202006171951.bak


    Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL2\pubs


Mode                 LastWriteTime         Length Name


----                 -------------         ------ ----


-a---          17/06/2020    19:51         524288 pubs_202006171951.bak


As you can see we have the databases backed up, each in their own directory :-)

## Double check all the instances

dbatoos works with multiple instances whereever there is a `-SqlInstance` parameter

Lets check the last time that these databases were backed up on both of the containers

In [10]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

ComputerName InstanceName SqlInstance  Database           LastFullBackup          LastDiffBackup


------------ ------------ -----------  --------           --------------          --------------   


localhost    MSSQLSERVER  b7ee5654561f AdventureWorks2017 2020-06-17 18:50:17.000 2019-12-21 14:42…


localhost    MSSQLSERVER  b7ee5654561f master             2020-06-17 18:50:18.000                  


localhost    MSSQLSERVER  b7ee5654561f model              2020-06-17 18:50:18.000                  


...

localhost    MSSQLSERVER  31f2db5347e2 master             2020-06-17 18:51:06.000                  


localhost    MSSQLSERVER  31f2db5347e2 model              2020-06-17 18:51:07.000                  


localhost    MSSQLSERVER  31f2db5347e2 msdb               2020-06-17 18:51:07.000                  


localhost    MSSQLSERVER  31f2db5347e2 Northwind          2020-06-17 18:51:08.000 2019-12-21 14:42…


localhost    MSSQLSERVER  31f2db5347e2 pubs               2020-06-17 18:51:09.000 2019-12-21 14:42…


# We want to use Ola Hallengren for our backups

Thats ok, dbatools supports many of the excellent community tooling including Ola Hallengrens solution.

You can install Ola Hallengren like this (except in containers)

````
$installDbaMaintenanceSolutionSplat = @{
    SqlInstance = $sql1
    InstallJobs = $true
    ReplaceExisting = $true
    LogToTable = $true
    OutputFileDirectory = '/var/opt/mssql/backups/SQL1'
    BackupLocation = '/var/opt/mssql/backups/SQL1'
    CleanupTime = 72
    Solution = 'All'
    Database = 'master'
}
Install-DbaMaintenanceSolution @installDbaMaintenanceSolutionSplat -Verbose
````

Lets have a look at the Agent Jobs on the Instance

In [11]:
Get-DbaAgentJob -SqlInstance $sql1 |Select Name

Name


----


CommandLog Cleanup


DatabaseBackup - SYSTEM_DATABASES - FULL


DatabaseBackup - USER_DATABASES - DIFF


...

DatabaseIntegrityCheck - USER_DATABASES


IndexOptimize - USER_DATABASES


sp_delete_backuphistory


sp_purge_jobhistory


The Beard is Important


Those look like Ola Jobs to me :-)

Lets run the jobs to back up the system and user databases

In [2]:
$Jobs = 'DatabaseBackup - SYSTEM_DATABASES - FULL' ,'DatabaseBackup - USER_DATABASES - FULL'
Start-DbaAgentJob -SqlInstance $sql1 -Job $Jobs

ComputerName           : localhost


InstanceName           : MSSQLSERVER


SqlInstance            : b7ee5654561f


Name                   : DatabaseBackup - SYSTEM_DATABASES - FULL


Category               : Database Maintenance


...

LastRunDate            : 17/06/2020 18:52:05


LastRunOutcome         : Failed


HasSchedule            : False


OperatorToEmail        : 


CreateDate             : 21/12/2019 14:35:24


Check the status of the jobs

In [3]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaAgentJob -SqlInstance $sql1 -Job $Jobs | Select SqlInstance, Name, CurrentRunStatus, LastRunOutCome

SqlInstance  Name                                     CurrentRunStatus LastRunOutcome


-----------  ----                                     ---------------- --------------


b7ee5654561f DatabaseBackup - SYSTEM_DATABASES - FULL             Idle         Failed


b7ee5654561f DatabaseBackup - USER_DATABASES - FULL               Idle         Failed


When the jobs have finished, lets have a look at the files.
We need to alter the file path to include the current container name from the results above

In [18]:
Get-DbaFile -SqlInstance $sql1 -Path '/var/opt/mssql/data/01a539677f71/AdventureWorks2017/FULL'

# Restores

Backups are all very well and good but what about restores?

**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason

In [19]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

ComputerName : localhost


InstanceName : MSSQLSERVER


SqlInstance  : b7ee5654561f


Database     : AdventureWorks2017


Status       : Dropped


...

ComputerName : localhost


InstanceName : MSSQLSERVER


SqlInstance  : b7ee5654561f


Database     : pubs


Status       : Dropped


In [21]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

All the databases have gone - How easy it is to restore them?

This easy :-)

One line of code

## First lets restore from the backups we took with dbatools in, with all files in the one directory

In [22]:
Write-Output "Starting Restoring"
Restore-DbaDatabase  -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 
Write-Output "Finished"

Starting Restoring


                                                                                                    
 Updating                                                                                           
    Progress                                                                                        
    [                                                                                        ]      
                                                                                                    
    Scanning Restore headers: 0/7                                                                   
                                                                                                    

                                                                                                    
 Updating                                                                                           
    Progress                                                                                        
    [                                                                                        ]      
                                                                                                    
    Scanning Restore headers: 0/7                                                                   
                                                                                                    

RESTORE HEADERONLY is terminating abnormally.


                                                                                                    
 Restoring pubs to localhost,15592 - Backup 1 of 1                                                  
    Processing                                                                                      
    [                                                                                        ]      
                                                                                                    

...

                       N'/var/opt/mssql/data/AdventureWorks2017_log.ldf',  NOUNLOAD,  STATS = 10}


RestoreDirectory     : /var/opt/mssql/data


WithReplace          : False


                                                                                                    
 Restoring AdventureWorks2017 to localhost,15592 - Backup 1 of 1                                    
    Progress: 100.00 %                                                                              
    [oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo]      
                                                                                                    

Finished


In [23]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

Name


----


pubs


Northwind


AdventureWorks2017


That was easy - WHat about if I have used Ola Hallengren?

AGAIN

**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason


In [ ]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

## Restore from Ola Halengren Backups 

- Use the path from the above files check with the new container name 

In [ ]:
Write-Output "Starting Restoring"
Restore-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/data/ca2f813564a4 -MaintenanceSolutionBackup
Write-Output "Finished"

In [ ]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

Thats all very well and good

## When were the databases restored?

and which backup was used?

In [24]:
Get-DbaDbRestoreHistory -SqlInstance $SQL1 -Last

BackupFinishDate : 17/06/2020 18:50:17


ComputerName     : localhost


Database         : AdventureWorks2017


Date             : 17/06/2020 19:13:05


From             : c:\var\opt\mssql\backups\SQL1\AdventureWorks2017_202006171950.bak


...

InstanceName     : MSSQLSERVER


RestoreType      : Database


SqlInstance      : b7ee5654561f


To               : /var/opt/mssql/data/pubs.mdf, /var/opt/mssql/data/pubs_log.ldf


Username         : sqladmin


# Don't forget to clean up

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up